In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/slh/icecube/')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,0"
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import polars as pl
import pandas as pd
import gc
import os
import numpy as np
from icecube.fastai_fix import *
from tqdm.notebook import tqdm
from icecube.data_train_v3 import RandomChunkSampler,LenMatchBatchSampler,IceCubeCache, DeviceDataLoader
from icecube.loss import loss, loss_vms, loss_comb
from icecube.models import EncoderWithDirectionReconstructionV22
from fastxtend.vision.all import EMACallback
from tqdm import tqdm

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-04-05 11:04:42 - get_logger - Writing log to logs/graphnet_20230405-110442.log


In [3]:
SELECTION = 'total'
OUT = 'V22FT3'
PATH = '../data/'

NUM_WORKERS = 24
SEED = 2023
bs = 1024 - 256
L = 192

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
os.makedirs(OUT, exist_ok=True)

In [4]:
def WrapperAdamW(param_groups,**kwargs):
    return OptimWrapper(param_groups,torch.optim.AdamW)




def load_matching_weights(model, weights_path):
    """
    Load model weights from a given path if they match, otherwise skip.
    Prints the number of matched and unmatched weights.

    :param model: The PyTorch model for which weights should be loaded.
    :param weights_path: The path to the saved weights file (.pth or .pt).
    """
    # Load the saved state dictionary
    saved_state_dict = torch.load(weights_path, map_location=torch.device('cpu'))

    # Get the model's state dictionary
    model_state_dict = model.state_dict()

    # Create a new state dictionary to store matching weights
    matching_state_dict = {}

    # Initialize counters for matched and unmatched weights
    matched_weights = 0
    unmatched_weights = 0

    # Iterate through the saved state dictionary
    for name, saved_weight in saved_state_dict.items():
        # Check if the name exists in the model's state dictionary and the shapes match
        if name in model_state_dict and model_state_dict[name].shape == saved_weight.shape:
            # If it matches, add it to the matching state dictionary
            matching_state_dict[name] = saved_weight
            matched_weights += 1
        else:
            print(f"Skipping weight: {name} - Shape mismatch or not found in model")
            unmatched_weights += 1

    # Update the model's state dictionary with the matching state dictionary
    model_state_dict.update(matching_state_dict)

    # Load the updated state dictionary into the model
    model.load_state_dict(model_state_dict)

    print(f"Matched weights: {matched_weights}")
    print(f"Unmatched weights: {unmatched_weights}")





In [5]:
fname = OUT

ds_train = IceCubeCache(PATH, mode='train', L=L, selection=SELECTION,reduce_size=0.125)
ds_train_len = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125, selection=SELECTION, mask_only=True)
sampler_train = RandomChunkSampler(ds_train_len, chunks=ds_train.chunks)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs, drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=NUM_WORKERS, persistent_workers=True))

ds_val = IceCubeCache(PATH, mode='eval', L=L, selection=SELECTION)
ds_val_len = IceCubeCache(PATH, mode='eval', L=L, selection=SELECTION, mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, batch_sampler=len_sampler_val,
            num_workers=0))


data = DataLoaders(dl_train,dl_val)
model = EncoderWithDirectionReconstructionV22(dim=384, dim_base=128, depth=8, head_size=32)
model.load_state_dict(torch.load('/opt/slh/icecube/hb_training_loop/V22FT2/models/model_5.pth'))
model = nn.DataParallel(model)
model = model.cuda()
learn = Learner(data, model,  path = OUT, loss_func=loss_comb,cbs=[GradientClip(3.0),CSVLogger(),EMACallback(), 
            SaveModelCallback(monitor='loss',comp=np.less,every_epoch=True),
            GradientAccumulation(n_acc=4)],
            metrics=[loss], opt_func=partial(WrapperAdamW,eps=1e-7)).to_fp16()




In [6]:
learn.fit_one_cycle(8, 
                    lr_max=2e-5,
                    wd=0.05, 
                    pct_start=0.01, 
                    div=25,
                    div_final=25,
                    moms=(0.95,0.95,0.95))

epoch,train_loss,valid_loss,loss,time
0,1.026204,1.034510,0.968071,3:00:07
1,1.025262,1.034127,0.967624,3:00:34
2,1.014138,1.033939,0.967433,3:00:36
3,1.018374,1.033434,0.966955,3:00:28


/opt/slh/icecube/icecube/data_train_v3.py:264: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  L = max(1,L // 16)
/opt/conda/lib/python3.7/site-packages/torch/cuda/nccl.py:51: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(inputs, collections.Container) or isinstance(inputs, torch.Tensor):
ERROR: Unexpected segmentation fault encountered in worker.
 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_846614/589553133.py:7 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_846614/589553133.py'                        │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/callback/schedule.py:119 in fit_one_cycle          │
│                                                                                                  │
│   116 │   lr_max = np.array([h['lr'] for h in self.opt.hypers])                                  │
│   117 │   scheds = {'lr': combined_cos(pct_start, lr_max/div, lr_max, lr_max/div_final),         │
│   118 │   │   │     'mom': combined_cos(pct_start, *(self.moms if moms is None else moms))}      │
│ ❱ 119 │   self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, sta   │
│   120                                                                                            │
│   121 # %% ../../nbs/14_callback.schedule.ipynb 50                                               │
│   122 @patch                                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/learner.py:256 in fit                              │
│                                                                                                  │
│   253 │   │   │   if wd is not None: self.opt.set_hypers(wd=wd)                                  │
│   254 │   │   │   self.opt.set_hypers(lr=self.lr if lr is None else lr)                          │
│   255 │   │   │   self.n_epoch = n_epoch                                                         │
│ ❱ 256 │   │   │   self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup   │
│   257 │                                                                                          │
│   258 │   def _end_cleanup(self): self.dl,self.xb,self.yb,self.pred,self.loss = None,(None,),(   │
│   259 │   def __enter__(self): self(_before_epoch); return self                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/learner.py:193 in _with_events                     │
│                                                                                                  │
│   190 │   │   self.xb,self.yb = b[:i],b[i:]                                                      │
│   191 │                                                                                          │
│   192 │   def _with_events(self, f, event_type, ex, final=noop):                                 │
│ ❱ 193 │   │   try: self(f'before_{event_type}');  f()                                            │
│   194 │   │   except ex: self(f'after_cancel_{event_type}')                                      │
│   195 │   │   self(f'after_{event_type}');  final()                                              │
│   196                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/learner.py:245 in _do_fit                          │
│                                                                                                  │
│   242 │   def _do_fit(self):                                                                     │
│   243 │   │   for epoch in range(self.n_epoch):                                                  │
│   244 │   │   │   self.epoch=epoch                         